### this should be moved to tsf-fmri github but im lazy. Makes the subject and run univariate input cope list for second level analysis in fsl feat

In [39]:
subjects = [f"SF{str(i).zfill(2)}" for i in range(2, 33) if i not in [15, 18]]
runs = range(1, 5)
copes = range(1, 9)
out_dir = 'C:/Users/jz421/Desktop/fslSecondLevelInputs.txt'
file_paths = []

for subject in subjects:
    for run in runs:
        for cope in copes:
            path = f"/mnt/munin2/Egner/Lab/StabFlex/Analysis/Univariate/{subject}/{subject}_RUN_{run}/{subject}_RUN_{run}.feat/stats/cope{cope}.nii.gz"
            file_paths.append(path)

# Joining the paths into a single string for the user to copy.
file_paths_str = "\n".join(file_paths)
file_paths_str

with open(output, 'w') as file:
    file.write(file_paths_str)

# dude this is such bad code, this is saving to desktop but the below cell is saving to the folder that we run the code in. Ugh you're not gonna get hired as a SWEE ever lol...


### makes ev matrix

In [38]:
import numpy as np

# Define the conditions
conditions = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75",
              "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]

# Define subjects and runs
subjects = [f"SF{str(i).zfill(2)}" for i in range(2, 33) if i not in [15, 18]]
runs = range(1, 5)

# Initialize the EV matrix with zeros
ev_matrix = np.zeros((32 * len(subjects), len(conditions))) # 4 runs * 8 copes * n subjects is how many rows in ev matrix we need, cuz that's how many inputs we have
print(len(ev_matrix))
# Function to get block order for a given subject and run
def get_block_order_for_run(subject, run):
    block_order_mapping = {
        "SF01": ["B,A", "D,C", "C,D", "B,A"],
        "SF02": ["A,B", "D,C", "D,A", "C,B"],
        "SF03": ["B,C", "A,D", "C,D", "B,A"],
        "SF04": ["B,C", "A,D", "D,A", "C,B"],
        "SF05": ["C,D", "B,A", "D,A", "C,B"],
        "SF06": ["A,B", "D,C", "B,C", "A,D"],
        "SF07": ["A,B", "D,C", "C,D", "B,A"],
        "SF08": ["A,B", "D,C", "D,A", "C,B"],
        "SF09": ["B,C", "A,D", "C,D", "B,A"],
        "SF10": ["B,C", "A,D", "D,A", "C,B"],
        "SF11": ["C,D", "B,A", "D,A", "C,B"],
        "SF12": ["A,B", "D,C", "B,C", "A,D"],
        "SF13": ["A,B", "D,C", "C,D", "B,A"],
        "SF14": ["A,B", "D,C", "D,A", "C,B"],
        "SF15": ["B,C", "A,D", "C,D", "B,A"],
        "SF16": ["B,C", "A,D", "D,A", "C,B"],
        "SF17": ["C,D", "B,A", "D,A", "C,B"],
        "SF18": ["A,B", "D,C", "A,B", "A,D"],
        "SF19": ["A,B", "D,C", "C,D", "B,A"],
        "SF20": ["A,B", "D,C", "D,A", "C,B"],
        "SF21": ["B,C", "A,D", "C,D", "B,A"],
        "SF22": ["B,C", "A,D", "D,A", "C,B"],
        "SF23": ["C,D", "B,A", "D,A", "C,B"],
        "SF24": ["A,B", "D,C", "B,C", "A,D"],
        "SF25": ["A,B", "D,C", "C,D", "B,A"],
        "SF26": ["A,B", "D,C", "D,A", "C,B"],
        "SF27": ["B,C", "A,D", "C,D", "B,A"],
        "SF28": ["B,C", "A,D", "D,A", "C,B"],
        "SF29": ["A,B", "D,C", "C,D", "B,A"],
        "SF30": ["A,B", "D,C", "B,C", "A,D"],
        "SF31": ["A,B", "D,C", "C,D", "B,A"],
        "SF32": ["A,B", "D,C", "D,A", "C,B"]
    }

    blocks = block_order_mapping.get(subject, ["", ""])[run - 1].split(',')
    return blocks

# Define the mapping of blocks to conditions
block_conditions = {
    "A": {"is": "i25s25", "ir": "i25r25", "cs": "c25s25", "cr": "c25r25"},
    "B": {"is": "i25s75", "ir": "i25r75", "cs": "c25s75", "cr": "c25r75"},
    "C": {"is": "i75s25", "ir": "i75r25", "cs": "c75s25", "cr": "c75r25"},
    "D": {"is": "i75s75", "ir": "i75r75", "cs": "c75s75", "cr": "c75r75"}
}

# Function to determine the condition for a given COPE, subject, and run
def determine_condition(subject, run, cope, conditions):
    print('run is: ', run)
    print('cope is: ', cope)
    block_order = get_block_order_for_run(subject, run)
    # so this will be two blocks
    print('block order: ', block_order)
    # Generate a list of conditions for each block
    run_conditions = []
    for block in block_order:
        for key in ["is", "ir", "cs", "cr"]:
            run_conditions.append(block_conditions[block][key])
            # should be two run conditions
    print("run conditions: ", run_conditions)

    # Sort these conditions based on their order in the 'conditions' list
    sorted_conditions = sorted(run_conditions, key=lambda x: conditions.index(x))
    print("sorted conditions: ", sorted_conditions)

    # Assign COPEs based on this sorted order. Eight conditions within the run.
    cope_condition = sorted_conditions[(cope - 1)]
    print("cope condition: ", cope_condition)
    return cope_condition

# Fill the EV matrix
row = 0
for subject in subjects:
    for run in runs:
        for cope in range(1, 9):
            condition = determine_condition(subject, run, cope, conditions)
            col = conditions.index(condition)
            ev_matrix[row, col] = 1
            row += 1


# Save the EV matrix to a file
np.savetxt("ev_matrix.txt", ev_matrix, fmt='%d')


928
run is:  1
cope is:  1
block order:  ['A', 'B']
run conditions:  ['i25s25', 'i25r25', 'c25s25', 'c25r25', 'i25s75', 'i25r75', 'c25s75', 'c25r75']
sorted conditions:  ['i25s25', 'i25s75', 'i25r25', 'i25r75', 'c25s25', 'c25s75', 'c25r25', 'c25r75']
cope condition:  i25s25
run is:  1
cope is:  2
block order:  ['A', 'B']
run conditions:  ['i25s25', 'i25r25', 'c25s25', 'c25r25', 'i25s75', 'i25r75', 'c25s75', 'c25r75']
sorted conditions:  ['i25s25', 'i25s75', 'i25r25', 'i25r75', 'c25s25', 'c25s75', 'c25r25', 'c25r75']
cope condition:  i25s75
run is:  1
cope is:  3
block order:  ['A', 'B']
run conditions:  ['i25s25', 'i25r25', 'c25s25', 'c25r25', 'i25s75', 'i25r75', 'c25s75', 'c25r75']
sorted conditions:  ['i25s25', 'i25s75', 'i25r25', 'i25r75', 'c25s25', 'c25s75', 'c25r25', 'c25r75']
cope condition:  i25r25
run is:  1
cope is:  4
block order:  ['A', 'B']
run conditions:  ['i25s25', 'i25r25', 'c25s25', 'c25r25', 'i25s75', 'i25r75', 'c25s75', 'c25r75']
sorted conditions:  ['i25s25', 'i25s7